In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import os
import random
import shutil

In [2]:
# Пути к данным
train_data_dir = 'data/'
not_agent_dir = 'data/not_inoagent/'
balanced_data_dir = 'data/balanced_data/'
balanced_not_agent_dir = os.path.join(balanced_data_dir, 'not_inoagent/')

required_not_agent_images = 100  # Столько же изображений нужно взять из класса "не иноагентов"

In [ ]:
# Создание директории для сбалансированных данных (иноагенты + не иноагенты)
if not os.path.exists(balanced_data_dir):
    os.makedirs(balanced_data_dir)
if not os.path.exists(balanced_not_agent_dir):
    os.makedirs(balanced_not_agent_dir)

# Копируем папки с изображениями иноагентов в новую сбалансированную директорию
for agent_dir in os.listdir(train_data_dir):
    agent_path = os.path.join(train_data_dir, agent_dir)
    if agent_dir.startswith('agent_'):
        shutil.copytree(agent_path, os.path.join(balanced_data_dir, agent_dir))

# Получаем все изображения "не иноагентов" и случайно выбираем нужное количество
all_not_agent_images = os.listdir(not_agent_dir)
selected_not_agent_images = random.sample(all_not_agent_images, required_not_agent_images)

# Копируем выбранные изображения в новую папку "balanced_not_agent"
for img in selected_not_agent_images:
    shutil.copy(os.path.join(not_agent_dir, img), os.path.join(balanced_not_agent_dir, img))

In [4]:
# Параметры модели
img_height, img_width = 300, 300  # Размер входных изображений для EfficientNet
batch_size = 8 #размер batch'a

# Создание генератора данных с аугментацией для обучения
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    #rotation_range=30,          # Увеличиваем диапазон вращения
    #width_shift_range=0.3,      # Увеличиваем сдвиги
    #height_shift_range=0.3,
    #shear_range=0.3,            # Увеличиваем сдвиг угла
    #zoom_range=0.3,             # Увеличиваем масштабирование
    #horizontal_flip=True,       # Используем горизонтальное отражение
    #brightness_range=[0.8, 1.2],  # Варьируем яркость изображений
    #fill_mode='nearest'
)


# Генераторы для тренировки и валидации на сбалансированных данных
train_generator = train_datagen.flow_from_directory(
    balanced_data_dir,         # Используем сбалансированную директорию
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # Для многоклассовой классификации
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    balanced_data_dir,         # Используем сбалансированную директорию
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Определение количества классов
num_classes = train_generator.num_classes

# Загрузка предобученной модели EfficientNetB0 без верхних слоев
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Замораживаем базовые слои
for layer in base_model.layers:
    layer.trainable = False

# Добавляем свои слои поверх базовой модели
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Глобальный Average Pooling
x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)  # Полносвязный слой
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='sigmoid')(x)  # Выходной слой с softmax для многоклассовой классификации #relu для многоклассовой

Found 160 images belonging to 2 classes.
Found 39 images belonging to 2 classes.


In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy']) #categorical_crossentropy для многоклассовой

for layer in base_model.layers[-100:]:
    layer.trainable = True

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10,
    callbacks=[early_stopping]
)

In [ ]:
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history_finetune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=5
)

model.save('inoagent_classifier.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

model = tf.keras.models.load_model('inoagent_classifier.h5')

img_path = 'test.jpg'
img = image.load_img(img_path, target_size=(224 * 4, 224 * 4))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0 

prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)

class_labels = train_generator.class_indices  
class_labels = {v: k for k, v in class_labels.items()}  

print(f"Класс: {class_labels[predicted_class[0]]}")